In [1]:
import numpy as np
import open3d as o3d
from scipy.optimize import least_squares
import pandas as pd
import os
import trimesh
import pyvista as pv
from scipy.spatial.transform import Rotation as R
from stl import mesh
import re
import copy
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
import random
from scipy.spatial.distance import cityblock

from scipy.spatial.distance import euclidean
random.seed(42) 
np.random.seed(42)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def translate_mesh_to_origin(mesh):
    # Calculate the centroid (center of mass)
    centroid = np.mean(np.asarray(mesh.vertices), axis=0)
    # Calculate the translation needed to move the centroid to the origin
    translation = -centroid
    # Apply the translation
    translated_vertices = np.asarray(mesh.vertices) + translation
    mesh.vertices = o3d.utility.Vector3dVector(translated_vertices)
    return mesh

def compute_centroid(point_cloud):
    return np.mean(np.asarray(point_cloud.points), axis=0)

def quaternion_from_vectors(vec1, vec2):
    """ Create a quaternion representing the rotation from vec1 to vec2 """
    v1_norm = vec1 / np.linalg.norm(vec1)
    v2_norm = vec2 / np.linalg.norm(vec2)
    d = np.dot(v1_norm, v2_norm)

    if np.isclose(d, -1.0):
        # Vectors are nearly opposite
        axis = np.cross(v1_norm, np.array([1, 0, 0]))
        if np.linalg.norm(axis) < 1e-8:  # If parallel to x-axis, use y-axis
            axis = np.cross(v1_norm, np.array([0, 1, 0]))
        axis = axis / np.linalg.norm(axis)
        return o3d.geometry.get_rotation_matrix_from_axis_angle(np.pi * axis)

    s = np.sqrt((1 + d) * 2)
    axis = np.cross(v1_norm, v2_norm)
    q = np.array([s * 0.5, axis[0] / s, axis[1] / s, axis[2] / s])
    return o3d.geometry.get_rotation_matrix_from_quaternion(q)

def align_mesh(mesh, target_axis1=[0, 0, 1], target_axis2=[0, 1, 0], target_axis3=[1, 0, 0]):
    vertices = np.asarray(mesh.vertices)
    centroid = vertices.mean(axis=0)
    centered_vertices = vertices - centroid

    try:
        u, s, vh = np.linalg.svd(centered_vertices, full_matrices=False)
    except np.linalg.LinAlgError:
        print("SVD did not converge, skipping alignment for this mesh.")
        return mesh

    # Align the first principal axis
    principal_axis1 = vh[0, :]
    rotation_matrix1 = quaternion_from_vectors(principal_axis1, target_axis1)
    rotated_vertices = np.dot(centered_vertices, rotation_matrix1.T)

    # Recalculate the principal axes and align the second principal axis
    u, s, vh = np.linalg.svd(rotated_vertices, full_matrices=False)
    principal_axis2 = vh[1, :]
    rotation_matrix2 = quaternion_from_vectors(principal_axis2, target_axis2)
    rotated_vertices = np.dot(rotated_vertices, rotation_matrix2.T)

    # Recalculate the principal axes again and align the third principal axis
    u, s, vh = np.linalg.svd(rotated_vertices, full_matrices=False)
    principal_axis3 = vh[2, :]
    rotation_matrix3 = quaternion_from_vectors(principal_axis3, target_axis3)
    rotated_vertices = np.dot(rotated_vertices, rotation_matrix3.T)

    mesh.vertices = o3d.utility.Vector3dVector(rotated_vertices + centroid)
    # mesh = final_orientation_adjustment(mesh, vertex_index=0, target_direction=[1 ,0, 0])
    mesh = translate_mesh_to_origin(mesh)
    
    return mesh

def rotate_randomly(mesh):
    """
    Rotates the mesh randomly around X, Y, and Z axes.
    
    Parameters:
    - mesh: The input mesh (open3d.geometry.TriangleMesh) to be rotated.
    
    Returns:
    - mesh: The rotated mesh.
    """
    # Generate three random angles between 0 and 360 degrees for rotation around each axis
    angles = np.random.uniform(low=0, high=2*np.pi, size=3)  # angles in radians

    # Create rotation matrices for X, Y, and Z axes
    R_x = mesh.get_rotation_matrix_from_xyz((angles[0], 0, 0))
    R_y = mesh.get_rotation_matrix_from_xyz((0, angles[1], 0))
    R_z = mesh.get_rotation_matrix_from_xyz((0, 0, angles[2]))

    # Combine the rotation matrices to apply the rotation in X, Y, and Z directions
    R = R_x @ R_y @ R_z

    # Rotate the mesh
    mesh.rotate(R, center=mesh.get_center())

    return mesh

In [3]:
#Read Meshes and sample point clouds

current_dir = os.getcwd()

stl_directory = os.path.join(current_dir,'ermetal')
pcd_directory = os.path.join(current_dir,'pcds_5k_random')

stl_file_paths = [os.path.join(stl_directory, f) for f in os.listdir(stl_directory) if f.endswith('.stl')]
stl_file_names = [os.path.basename(f) for f in stl_file_paths]

pcd_file_paths = [os.path.join(pcd_directory, f) for f in os.listdir(stl_directory) if f.endswith('.stl')]
pcd_file_paths = [f.replace('.stl', '.pcd') for f in pcd_file_paths]


def load_point_cloud(stl_file_path, pcd_file_path=None, visualize=False, cube_side_length=1000):
    if pcd_file_path and os.path.exists(pcd_file_path):
        point_cloud = o3d.io.read_point_cloud(pcd_file_path)        
        mesh = o3d.io.read_triangle_mesh(stl_file_path)
        mesh = rotate_randomly(mesh)
    else:
        mesh = o3d.io.read_triangle_mesh(stl_file_path)
        mesh = rotate_randomly(mesh)
        point_cloud = mesh.sample_points_uniformly(number_of_points=5_000)
        if pcd_file_path:
            o3d.io.write_point_cloud(pcd_file_path, point_cloud)
    if visualize:
        visualize_point_cloud(point_cloud)
    return point_cloud,mesh

# Usage example
pcds_o3d = []
meshes = []
for i in range(len(pcd_file_paths)):
    pcd,mesh = load_point_cloud(stl_file_paths[i], pcd_file_paths[i], False, cube_side_length=1000)
    meshes.append(mesh)
    pcds_o3d.append(pcd)

In [4]:
import numpy as np
import open3d as o3d
import gudhi as gd
def compute_betti_numbers_from_open3d_mesh(mesh):
    """
    Computes Betti numbers for a given Open3D mesh.

    Parameters:
    - mesh: An Open3D mesh object.

    Returns:
    - A list of Betti numbers [b0, b1, b2, ...] representing the topology of the mesh.
    """

    # Initialize the simplicial complex
    simplicial_complex = gd.SimplexTree()

    # Add vertices
    for i, vertex in enumerate(np.asarray(mesh.vertices)):
        simplicial_complex.insert([i], filtration=0)

    # Add edges and triangles (faces)
    for face in np.asarray(mesh.triangles):
        # Add edges
        simplicial_complex.insert(sorted([face[0], face[1]]), filtration=0)
        simplicial_complex.insert(sorted([face[1], face[2]]), filtration=0)
        simplicial_complex.insert(sorted([face[2], face[0]]), filtration=0)
        # Add triangle
        simplicial_complex.insert(sorted(face), filtration=0)

    # Compute persistence (homology in various dimensions)
    persistence = simplicial_complex.persistence()

    # Extract Betti numbers
    betti_numbers = simplicial_complex.betti_numbers()

    return betti_numbers

In [5]:
def compute_normals_variance(mesh):
    # Access vertices and triangles
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    # Calculate triangle (cell) normals
    triangle_normals = np.abs(np.cross(vertices[triangles[:, 1]] - vertices[triangles[:, 0]],
                                vertices[triangles[:, 2]] - vertices[triangles[:, 0]]))
    # triangle_normals = np.cross(vertices[triangles[:, 1]] - vertices[triangles[:, 0]],
    #                             vertices[triangles[:, 2]] - vertices[triangles[:, 0]])
    norms = np.linalg.norm(triangle_normals, axis=1)

    # Filter out invalid triangles and normals
    zero_vector_threshold = 1e-20
    valid_triangles_mask = norms > zero_vector_threshold
    valid_normals = triangle_normals[valid_triangles_mask]

    # Calculate areas of valid triangles
    valid_triangle_areas = norms[valid_triangles_mask] * 0.5

    # Normalize the valid normals
    valid_normals_normalized = valid_normals / np.linalg.norm(valid_normals, axis=1, keepdims=True)

    # Check and remove NaN values in normalized normals
    nan_mask = ~np.isnan(valid_normals_normalized).any(axis=1)
    valid_normals_normalized = valid_normals_normalized[nan_mask]
    valid_triangle_areas = valid_triangle_areas[nan_mask]

    # Ensure there are still valid normals left after filtering
    if valid_normals_normalized.size == 0:
        return np.nan, np.nan, 0

    # Compute weighted normals
    normals_weighted = valid_normals_normalized * valid_triangle_areas[:, np.newaxis]

    # Compute mean normal
    total_area = np.sum(valid_triangle_areas)
    mean_normal = np.sum(normals_weighted, axis=0) / total_area
    # print('mean normal', mean_normal)
    # print(np.linalg.norm(mean_normal))
    normalized_mean_normal = mean_normal / np.linalg.norm(mean_normal)
    normals_weighted_normalized =  normals_weighted / np.linalg.norm(normals_weighted, axis=1, keepdims=True)
    # Compute distances and weighted mean distance
    distances = np.linalg.norm(normals_weighted_normalized - normalized_mean_normal, axis=1)
    weighted_mean_distance = np.sum(distances * valid_triangle_areas) / total_area

    # Calculate angular differences
    dot_products = np.dot(normals_weighted_normalized, normalized_mean_normal)
    dot_products = np.clip(dot_products, -1.0, 1.0)
    angular_differences_rad = np.arccos(dot_products)
    angular_differences_deg = np.degrees(angular_differences_rad)

    deg_diff = np.sum(angular_differences_deg * valid_triangle_areas) / total_area
    rad_diff = np.sum(angular_differences_rad * valid_triangle_areas) / total_area
    
    return weighted_mean_distance, rad_diff, deg_diff, total_area

In [6]:
# def compute_fpfh_features(point_cloud, radius):
#     point_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius*2, max_nn=30))

#     fpfh = o3d.pipelines.registration.compute_fpfh_feature(
#         point_cloud,
#         o3d.geometry.KDTreeSearchParamHybrid(radius=radius*5, max_nn=50)
#     )
#     normals = np.asarray(point_cloud.normals)
#     normal_variance_var = np.var(normals, axis=0)
#     return fpfh,normal_variance_var[0], normal_variance_var[1], normal_variance_var[2]
# def aggregate_fpfh_features(fpfh):
#     return [np.mean(np.asarray(fpfh.data), axis=1),np.std(np.asarray(fpfh.data), axis=1)]
def compute_fpfh_features(point_cloud, radius):
    point_cloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=radius*3, max_nn=100))

    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        point_cloud,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius*5, max_nn=80)
    )
    normals = np.asarray(point_cloud.normals)
    normal_variance_var = np.var(normals, axis=0)
    return fpfh,normal_variance_var[0], normal_variance_var[1], normal_variance_var[2]
def aggregate_fpfh_features(fpfh):
    return [np.mean(np.asarray(fpfh.data), axis=1),np.std(np.asarray(fpfh.data), axis=1)]


In [7]:
def scale_point_cloud(pc, cube_side_length):
    obb = pc.get_oriented_bounding_box()
    max_extent = max(obb.extent.tolist())  # Get the largest dimension of the OBB

    # Calculate scale factor to fit the point cloud within the cube
    scale_factor = cube_side_length / max_extent
    
    # Rotation matrix to align the OBB with the axes, expanded to 4x4
    R = np.eye(4)
    R[:3, :3] = obb.R
    
    # Scaling matrix
    S = np.diag([scale_factor, scale_factor, scale_factor, 1])
    
    # Translation matrix to move OBB center to origin, expanded to 4x4
    T = np.eye(4)
    T[:3, 3] = -obb.center

    # Inverse translation to move back after scaling and rotation
    T_inv = np.eye(4)
    T_inv[:3, 3] = obb.center
    
    # Compute the full transformation matrix
    # The correct order is: translate to origin, rotate to align with axes, scale, undo rotation, translate back
    transformation_matrix = T_inv @ R.T @ S @ R @ T
    
    # Apply the transformation to the point cloud
    pc.transform(transformation_matrix)
    
    return pc

In [8]:
def compute_similarity(source, mesh, voxel_size):
     
    obb = mesh.get_oriented_bounding_box()
    extents = obb.extent
    sorted_extents = np.sort(extents)  # Sorts the extents in ascending order by default
    metrics = sorted_extents   
    betti_numbers = compute_betti_numbers_from_open3d_mesh(mesh)
    weighted_mean_distance, rad_diff,deg_diff, total_area = compute_normals_variance(mesh)
    
    source_copy = copy.deepcopy(source)
    source = scale_point_cloud(source,1000)

    source=source.voxel_down_sample(voxel_size)
    fpfh, normal_variance_var_x, normal_variance_var_y,normal_variance_var_z  =compute_fpfh_features(source, voxel_size)
    fpfh_mean, fpfh_std= aggregate_fpfh_features(fpfh)

    std_fpfh_mean = np.std(fpfh_mean)
    mean_fpfh_mean = np.mean(fpfh_mean)
    std_fpfh_std = np.std(fpfh_std)
    mean_fpfh_std = np.mean(fpfh_std)

    

    metrics = np.concatenate((metrics,[betti_numbers[1],rad_diff,weighted_mean_distance]))
    metrics = np.concatenate((metrics,[std_fpfh_mean,std_fpfh_std,mean_fpfh_std]))
    metrics = np.concatenate((metrics,[normal_variance_var_x, normal_variance_var_y,normal_variance_var_z]))
    return metrics


In [9]:
scores_dict = {}
# Iterate over each point cloud and its corresponding mesh
for (pc, mesh), pc_index in zip(zip(pcds_o3d, meshes), range(len(pcds_o3d))):
    pc_name = stl_file_names[pc_index]  # Adjust the split if needed
    pc_scores = []  # Initialize scores as an empty list

    scores = compute_similarity(pc, mesh, 5)
    # scores = compute_similarity(pc, shape, int(shape_path))  
    
    # Append each score in the rmse list to pc_scores
    # pc_scores.extend(scores)  # Assuming scores is a list of rmse values
    # Print results
    print(f"PointCloud {pc_name} is done with the score: {scores}")
    # Store the scores for the current point cloud in the main dictionary
    scores_dict[pc_name] = scores

# [mesh_center_mean, distance_normal, angle_normal,euclidean_norm_std,manhattan_norm_std,euclidean_norm_mean,manhattan_norm_mean, hausdorff, rmse, centre_std, average_density, normal_var_x, normal_var_y, std_fpfh_std, mean_fpfh_std]


PointCloud 001_2020_Teklifid_10.stl is done with the score: [26.29311146 53.2397749  54.29333923  2.          0.48203455  0.47647113
 31.19799983 11.84602371 19.23677889  0.18413926  0.10619927  0.70954022]
PointCloud 001_2020_Teklifid_11.stl is done with the score: [5.26141376e+01 1.68614148e+02 3.60008835e+02 1.00000000e+01
 4.01025212e-01 3.96076604e-01 1.18709207e+01 2.52742765e+00
 8.20610037e+00 4.73408653e-01 2.42626458e-01 2.83862799e-01]
PointCloud 001_2020_Teklifid_12.stl is done with the score: [ 80.97962288 247.91778331 281.22182672  17.           0.52545751
   0.51546987   7.90954874   2.66050281  10.52227184   0.28497836
   0.29598073   0.41901673]
PointCloud 001_2020_Teklifid_13.stl is done with the score: [1.38196140e+02 2.03114622e+02 2.42644094e+02 1.90000000e+01
 4.56228273e-01 4.50035769e-01 1.59052427e+01 6.70475703e+00
 1.43046603e+01 2.54086202e-01 1.73343961e-01 5.72468207e-01]
PointCloud 001_2020_Teklifid_1306.stl is done with the score: [ 25.27769509  80.70837

In [12]:
import pickle

with open('scores_dict.pkl','wb') as file:
    pickle.dump(scores_dict,file)
with open('distance_dict.pkl','wb') as file:
    pickle.dump(distance_dict,file)
with open('ranking_dict.pkl','wb') as file:
    pickle.dump(ranking_dict,file)


In [3]:
# # import pickle

# import pickle
# with open('scores_dict_reduced.pkl','rb') as file:
#     scores_dict =pickle.load(file)

In [11]:
#### Normalizing the scores in scores_dict
def normalize_scores(scores_dict):
    # Convert scores to numpy array for normalization
    scores_array = np.array(list(scores_dict.values()))

    # Create a MinMaxScaler
    scaler = RobustScaler()

    # Fit and transform the data
    normalized_scores = scaler.fit_transform(scores_array)

    # Convert normalized scores back to the dictionary format
    normalized_scores_dict = {key: list(value) for key, value in zip(scores_dict.keys(), normalized_scores)}

    return normalized_scores_dict

def create_ranking_dict(distance_dict):
    ranking_dict = {}

    for pc_name, distances in distance_dict.items():
        ranking = [name for name, _ in distances]
        ranking_dict[pc_name] = ranking

    return ranking_dict


# Function to compute Manhattan distances
def compute_manhattan_distances(scores_dict):
    distance_dict = {}

    for pc_name, pc_scores in scores_dict.items():
        distances = []
        for other_pc_name, other_pc_scores in scores_dict.items():
            if pc_name != other_pc_name:
                # Calculate Manhattan distance
                distance = cityblock(pc_scores, other_pc_scores)
                distances.append((other_pc_name, distance))
        # Sort based on distance
        distances.sort(key=lambda x: x[1])
        distance_dict[pc_name] = distances

    return distance_dict
    


def compute_euclidean_distances(scores_dict):
    distance_dict = {}

    for pc_name, pc_scores in scores_dict.items():
        distances = []
        for other_pc_name, other_pc_scores in scores_dict.items():
            if pc_name != other_pc_name:
                # Calculate Euclidean distance
                distance = euclidean(pc_scores, other_pc_scores)
                distances.append((other_pc_name, distance))
        # Sort based on distance
        distances.sort(key=lambda x: x[1])
        distance_dict[pc_name] = distances

    return distance_dict


# Your existing code to normalize and rank the scores
normalized_dict = normalize_scores(scores_dict)

# Compute Manhattan distances
distance_dict = compute_manhattan_distances(scores_dict)
# distance_dict = compute_euclidean_distances(normalized_dict)

# Create the ranking dictionary
ranking_dict = create_ranking_dict(distance_dict)

a = ''
# Read the Excel file
curr=os.getcwd()

df = pd.read_excel(curr+'\\labelled_files.xlsx')

# Loop through the dataframe
for index, row in df.iterrows():
    stl_file_name = row['part_id_stl_file']
    top_choice = row['top_choice_stl_file']

    # Check if stl_file_name is in ranking_dict
    if stl_file_name in ranking_dict:
        ranking = ranking_dict[stl_file_name]
        
        # Check if the top choice matches the first rank in ranking_dict
        if ranking[0] == top_choice:
            # print(f"{stl_file_name}: matching file {top_choice} has rank 1")
            pass
        else:
            
            # Find the rank of the top choice in the ranking list
            if top_choice in ranking:
                rank = ranking.index(top_choice) + 1
                if rank <= 5:
                    # print(f"{stl_file_name}: matching file {top_choice} has rank {rank}")
                    pass
                else:
                    print(f"{stl_file_name}: matching file {top_choice} is beyond top 5, ranked at {rank}")
                    pass
            else:
                pass
                # print(f"{stl_file_name}: matching file {top_choice} is not ranked")
    else:
        print(f"{stl_file_name} is not in the ranking dictionary")

#ALIGNMENT AND VISUALIZATION FUNCTIONS 

def quaternion_from_vectors(vec1, vec2):
    """ Create a quaternion representing the rotation from vec1 to vec2 """
    v1_norm = vec1 / np.linalg.norm(vec1)
    v2_norm = vec2 / np.linalg.norm(vec2)
    d = np.dot(v1_norm, v2_norm)

    if np.isclose(d, -1.0):
        # Vectors are nearly opposite
        axis = np.cross(v1_norm, np.array([1, 0, 0]))
        if np.linalg.norm(axis) < 1e-8:  # If parallel to x-axis, use y-axis
            axis = np.cross(v1_norm, np.array([0, 1, 0]))
        axis = axis / np.linalg.norm(axis)
        return o3d.geometry.get_rotation_matrix_from_axis_angle(np.pi * axis)

    s = np.sqrt((1 + d) * 2)
    axis = np.cross(v1_norm, v2_norm)
    q = np.array([s * 0.5, axis[0] / s, axis[1] / s, axis[2] / s])
    return o3d.geometry.get_rotation_matrix_from_quaternion(q)



def visualize_mesh_with_triangles(mesh):
    """Visualizes a mesh with colored triangles and black lines along the edges of the triangles."""

    # mesh = translate_mesh_to_origin(mesh)
    vertex_colors = np.full((len(np.asarray(mesh.vertices)), 3), [0.8, 0.8, 0.8])
    mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)
    lines = [[triangle[0], triangle[1]] for triangle in np.asarray(mesh.triangles)] + \
            [[triangle[1], triangle[2]] for triangle in np.asarray(mesh.triangles)] + \
            [[triangle[2], triangle[0]] for triangle in np.asarray(mesh.triangles)]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.asarray(mesh.vertices)),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.paint_uniform_color([0, 0, 0])
    return [mesh, line_set]



#ALIGNMENT AND VISUALIZATION FUNCTIONS 

def quaternion_from_vectors(vec1, vec2):
    """ Create a quaternion representing the rotation from vec1 to vec2 """
    v1_norm = vec1 / np.linalg.norm(vec1)
    v2_norm = vec2 / np.linalg.norm(vec2)
    d = np.dot(v1_norm, v2_norm)

    if np.isclose(d, -1.0):
        # Vectors are nearly opposite
        axis = np.cross(v1_norm, np.array([1, 0, 0]))
        if np.linalg.norm(axis) < 1e-8:  # If parallel to x-axis, use y-axis
            axis = np.cross(v1_norm, np.array([0, 1, 0]))
        axis = axis / np.linalg.norm(axis)
        return o3d.geometry.get_rotation_matrix_from_axis_angle(np.pi * axis)

    s = np.sqrt((1 + d) * 2)
    axis = np.cross(v1_norm, v2_norm)
    q = np.array([s * 0.5, axis[0] / s, axis[1] / s, axis[2] / s])
    return o3d.geometry.get_rotation_matrix_from_quaternion(q)

def align_mesh(mesh, target_axis1=[0, 0, 1], target_axis2=[0, 1, 0], target_axis3=[1, 0, 0]):
    vertices = np.asarray(mesh.vertices)
    centroid = vertices.mean(axis=0)
    centered_vertices = vertices - centroid

    try:
        u, s, vh = np.linalg.svd(centered_vertices, full_matrices=False)
    except np.linalg.LinAlgError:
        print("SVD did not converge, skipping alignment for this mesh.")
        return mesh

    # Align the first principal axis
    principal_axis1 = vh[0, :]
    rotation_matrix1 = quaternion_from_vectors(principal_axis1, target_axis1)
    rotated_vertices = np.dot(centered_vertices, rotation_matrix1.T)

    # Recalculate the principal axes and align the second principal axis
    u, s, vh = np.linalg.svd(rotated_vertices, full_matrices=False)
    principal_axis2 = vh[1, :]
    rotation_matrix2 = quaternion_from_vectors(principal_axis2, target_axis2)
    rotated_vertices = np.dot(rotated_vertices, rotation_matrix2.T)

    # Recalculate the principal axes again and align the third principal axis
    u, s, vh = np.linalg.svd(rotated_vertices, full_matrices=False)
    principal_axis3 = vh[2, :]
    rotation_matrix3 = quaternion_from_vectors(principal_axis3, target_axis3)
    rotated_vertices = np.dot(rotated_vertices, rotation_matrix3.T)

    mesh.vertices = o3d.utility.Vector3dVector(rotated_vertices + centroid)

 
    return mesh
def visualize_meshes(filenames, directory_path):
    geometries_to_show = []
    current_x_position = 50
    spacing_factor = 3.5  # Adjust this factor to control spacing between the meshes

    for i, filename in enumerate(filenames):
        full_path = os.path.join(directory_path, filename)
        mesh_open3d = o3d.io.read_triangle_mesh(full_path)
        mesh_open3d = rotate_randomly(mesh_open3d)
        # Compute the axis-aligned bounding box to get the x dimension
        aabb = mesh_open3d.get_axis_aligned_bounding_box()
        x_dim = aabb.get_extent()[0]
        # For the first mesh (target mesh), we start at (0,0,0) and account for its size in spacing
        if i == 0:
            mesh_open3d.translate(-aabb.get_center())
        else:
            mesh_open3d.translate((current_x_position, 0, 0) - aabb.get_center())
        geometries_to_show.extend(visualize_mesh_with_triangles(mesh_open3d))
        current_x_position += x_dim * spacing_factor  # Update the x position
    
    o3d.visualization.draw_geometries(geometries_to_show)

#VISUALIZE BAD EXAMPLES, THE FIRST ONE IS THE TARGET PART, SECOND ONE IS THE ERMETAL TOP CHOICE, AND THE REST ARE ALGORITHM CHOICES

current_dir = os.getcwd()
# loading ermetal stl meshes
# stl_directory = os.path.join(current_dir,'updated_stl')
stl_directory = os.path.join(current_dir,'ermetal')

pcd_directory = os.path.join(current_dir,'pcd_5k')
# Initialize a list to store filenames for visualization along with their top choices
files_to_visualize_with_top_choice = []

# Loop through the dataframe
for index, row in df.iterrows():
    stl_file_name = row['part_id_stl_file']
    top_choice = row['top_choice_stl_file']

    if stl_file_name in ranking_dict:
        ranking = ranking_dict[stl_file_name]

        if ranking[0] != top_choice and top_choice in ranking:
            rank = ranking.index(top_choice) + 1
            if rank > 5:
                # Add the STL file, its top choice, and its top 5 similar parts to the visualization list
                files_to_visualize_with_top_choice.append((stl_file_name, top_choice, ranking[:3]))
    else:
        print(f"{stl_file_name} is not in the ranking dictionary")

# Visualize the collected STL files, their top choices, and their top 5 similar parts
directory_path = stl_directory  # Replace with the actual path to your STL files
for stl_file, top_choice, top_matches in files_to_visualize_with_top_choice:
    print('target file',stl_file,': ',scores_dict[stl_file],"\n")
    print('ermetal top choice', top_choice, ": ", scores_dict[top_choice],'\n')

    # print('target file',stl_file,': ',normalized_dict[stl_file],"\n")
    # print('ermetal top choice', top_choice, ": ", normalized_dict[top_choice],'\n')
    
    for i in range(len(top_matches)):
        print('algorithm top match #',i+1, top_matches[i],": ",scores_dict[top_matches[i]],'\n')
        # print('algorithm top match #',i+1, top_matches[i],": ",normalized_dict[top_matches[i]],'\n')
        
    visualize_filenames = [stl_file, top_choice] + top_matches
    visualize_meshes(visualize_filenames, directory_path)

# import pandas as pd
# columns = [
#     "part_id", "score_type", "rank","sorted_dim_a", "sorted_dim_b", "sorted_dim_c", "betti_number",
#     "rad_diff","weighted_mean_distance","std_fpfh_mean", "std_fpfh_std",
#     "mean_fpfh_std","normal_variance_var_x", "normal_variance_var_y"
# ]



# Initialize the DataFrame with specified columns
# df_scores = pd.DataFrame(columns=columns)

# for item in files_to_visualize_with_top_choice:
#     stl_file, top_choice, top_matches = item
#     parts_to_process = [stl_file, top_choice] + top_matches
    
#     for part_id in parts_to_process:
#         if part_id in scores_dict:
#             scores_list = scores_dict[part_id]
#             if isinstance(scores_list, list) and len(scores_list) == len(columns) - 2:
#                 scores = dict(zip(columns[3:], scores_list))
#                 if stl_file != part_id:
#                     rank = ranking_dict[stl_file].index(part_id) +1
#                 else:
#                     rank = 0
#                 row = {
#                     "part_id": part_id,
#                     "score_type": "target" if part_id == stl_file else ("ermetal_choice" if part_id == top_choice else "top_match"),
#                     "rank":rank,
#                     **scores
#                 }
#                 df_scores = df_scores.append(row, ignore_index=True)
#             else:
#                 print(f"Unexpected format or length for scores of {part_id}.")
#         else:
#             print(f"Scores for {part_id} not found in scores_dict.")
    
#     # Append an empty row after each set
#     # You can choose to leave the dictionary empty or include only the column names with empty values
#     empty_row = {col: "" for col in columns}
#     df_scores = df_scores.append(empty_row, ignore_index=True)

# # Write the DataFrame to an Excel file
# excel_path = 'D:\\0) Main Similarity\\Final Similarity\\scores_reduced.xlsx'
# with pd.ExcelWriter(excel_path, engine='xlsxwriter') as writer:
#     df_scores.to_excel(writer, sheet_name='Scores', index=False)
# print(f"Scores written to {excel_path}")


# 099_2022_Teklifid_4141.stl: matching file 099_2022_Teklifid_4131.stl is beyond top 5, ranked at 13
# 997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 12
# 142_2020_Teklifid_1374.stl: matching file 236_2020_Teklifid_1998.stl is beyond top 5, ranked at 58
# 228_2020_Teklifid_1967.stl: matching file 228_2020_Teklifid_1968.stl is beyond top 5, ranked at 27
# 543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 44
# 142_2020_Teklifid_1412.stl: matching file 142_2020_Teklifid_1415.stl is beyond top 5, ranked at 9
# 997_2021_Teklifid_3924.stl: matching file 997_2021_Teklifid_3940.stl is beyond top 5, ranked at 7
# 053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 176
# 093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 79
# 122_2020_Teklifid_1091.stl: matching file 122_2020_Teklifid_1093.stl is beyond top 5, ranked at 7
# 098_2022_Teklifid_4135.stl: matching file 098_2022_Teklifid_4134.stl is beyond top 5, ranked at 12
# 543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 20
# 142_2020_Teklifid_1837.stl: matching file 142_2020_Teklifid_1393.stl is beyond top 5, ranked at 19
# 121_2020_Teklifid_1019.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 28
# 184_2022_Teklifid_4342.stl: matching file 007_2020_Teklifid_186.stl is beyond top 5, ranked at 6
# 120_2020_Teklifid_994.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 63
# 037_2021_Teklifid_2457.stl: matching file 099_2022_Teklifid_4133.stl is beyond top 5, ranked at 13


# 099_2022_Teklifid_4141.stl: matching file 099_2022_Teklifid_4131.stl is beyond top 5, ranked at 74
# 997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 14
# 142_2020_Teklifid_1374.stl: matching file 236_2020_Teklifid_1998.stl is beyond top 5, ranked at 73
# 228_2020_Teklifid_1967.stl: matching file 228_2020_Teklifid_1968.stl is beyond top 5, ranked at 53
# 543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 112
# 053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 200
# 093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 95
# 543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 7
# 142_2020_Teklifid_1837.stl: matching file 142_2020_Teklifid_1393.stl is beyond top 5, ranked at 13
# 121_2020_Teklifid_1019.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 33
# 120_2020_Teklifid_994.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 95
# 037_2021_Teklifid_2457.stl: matching file 099_2022_Teklifid_4133.stl is beyond top 5, ranked at 52
# 121_2020_Teklifid_1010.stl: matching file 016_2020_Teklifid_262.stl is beyond top 5, ranked at 7
# 140_2020_Teklifid_1212.stl: matching file 140_2020_Teklifid_1211.stl is beyond top 5, ranked at 23


# 099_2022_Teklifid_4141.stl: matching file 099_2022_Teklifid_4131.stl is beyond top 5, ranked at 32
# 036_2020_Teklifid_404.stl: matching file 309_2021_Teklifid_3029.stl is beyond top 5, ranked at 11
# 997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 16
# 142_2020_Teklifid_1374.stl: matching file 236_2020_Teklifid_1998.stl is beyond top 5, ranked at 32
# 228_2020_Teklifid_1967.stl: matching file 228_2020_Teklifid_1968.stl is beyond top 5, ranked at 42
# 543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 41
# 997_2021_Teklifid_3924.stl: matching file 997_2021_Teklifid_3940.stl is beyond top 5, ranked at 13
# 053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 181
# 093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 109
# 098_2022_Teklifid_4135.stl: matching file 098_2022_Teklifid_4134.stl is beyond top 5, ranked at 11
# 543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 21
# 142_2020_Teklifid_1837.stl: matching file 142_2020_Teklifid_1393.stl is beyond top 5, ranked at 37
# 120_2020_Teklifid_994.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 9
# 037_2021_Teklifid_2457.stl: matching file 099_2022_Teklifid_4133.stl is beyond top 5, ranked at 22

# 017_2020_Teklifid_286.stl: matching file 017_2020_Teklifid_290.stl is beyond top 5, ranked at 17
# 121_2020_Teklifid_1034.stl: matching file 121_2020_Teklifid_1035.stl is beyond top 5, ranked at 16
# 036_2020_Teklifid_404.stl: matching file 309_2021_Teklifid_3029.stl is beyond top 5, ranked at 8
# 997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 55
# 142_2020_Teklifid_1374.stl: matching file 236_2020_Teklifid_1998.stl is beyond top 5, ranked at 13
# 228_2020_Teklifid_1967.stl: matching file 228_2020_Teklifid_1968.stl is beyond top 5, ranked at 11
# 543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 101
# 142_2020_Teklifid_1412.stl: matching file 142_2020_Teklifid_1415.stl is beyond top 5, ranked at 11
# 997_2021_Teklifid_3924.stl: matching file 997_2021_Teklifid_3940.stl is beyond top 5, ranked at 11
# 053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 170
# 093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 164
# 004_2020_Teklifid_342.stl: matching file 004_2020_Teklifid_343.stl is beyond top 5, ranked at 14
# 543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 18
# 142_2020_Teklifid_1837.stl: matching file 142_2020_Teklifid_1393.stl is beyond top 5, ranked at 6
# 007_2021_Teklifid_2365.stl: matching file 007_2021_Teklifid_2337.stl is beyond top 5, ranked at 18
# 184_2022_Teklifid_4342.stl: matching file 007_2020_Teklifid_186.stl is beyond top 5, ranked at 8
# 120_2020_Teklifid_994.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 12
# 098_2022_Teklifid_4112.stl: matching file 098_2022_Teklifid_4113.stl is beyond top 5, ranked at 10
# 037_2021_Teklifid_2457.stl: matching file 099_2022_Teklifid_4133.stl is beyond top 5, ranked at 35


# 099_2022_Teklifid_4141.stl: matching file 099_2022_Teklifid_4131.stl is beyond top 5, ranked at 31
# 036_2020_Teklifid_404.stl: matching file 309_2021_Teklifid_3029.stl is beyond top 5, ranked at 10
# 997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 47
# 142_2020_Teklifid_1374.stl: matching file 236_2020_Teklifid_1998.stl is beyond top 5, ranked at 45
# 228_2020_Teklifid_1967.stl: matching file 228_2020_Teklifid_1968.stl is beyond top 5, ranked at 43
# 543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 54
# 997_2021_Teklifid_3924.stl: matching file 997_2021_Teklifid_3940.stl is beyond top 5, ranked at 15
# 053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 180
# 093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 101
# 098_2022_Teklifid_4135.stl: matching file 098_2022_Teklifid_4134.stl is beyond top 5, ranked at 12
# 543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 21
# 142_2020_Teklifid_1837.stl: matching file 142_2020_Teklifid_1393.stl is beyond top 5, ranked at 39
# 997_2021_Teklifid_3940.stl: matching file 997_2021_Teklifid_3932.stl is beyond top 5, ranked at 6
# 184_2022_Teklifid_4342.stl: matching file 007_2020_Teklifid_186.stl is beyond top 5, ranked at 28
# 120_2020_Teklifid_994.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 19
# 037_2021_Teklifid_2457.stl: matching file 099_2022_Teklifid_4133.stl is beyond top 5, ranked at 44

099_2022_Teklifid_4141.stl: matching file 099_2022_Teklifid_4131.stl is beyond top 5, ranked at 32
036_2020_Teklifid_404.stl: matching file 309_2021_Teklifid_3029.stl is beyond top 5, ranked at 8
997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 34
142_2020_Teklifid_1374.stl: matching file 236_2020_Teklifid_1998.stl is beyond top 5, ranked at 44
228_2020_Teklifid_1967.stl: matching file 228_2020_Teklifid_1968.stl is beyond top 5, ranked at 45
543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 49
997_2021_Teklifid_3924.stl: matching file 997_2021_Teklifid_3940.stl is beyond top 5, ranked at 15
053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 182
093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 111
098_2022_Teklifid_4135.stl: matching file 098_2022_Teklifid_4134.stl is beyond top 5, ranked at 11
543_2021_Te

KeyboardInterrupt: 

In [71]:
# import pickle
# with open('ranking_dict_reduced.pkl','wb') as file:
#     pickle.dump(ranking_dict,file)

In [52]:
scores_dict['099_2022_Teklifid_4141.stl'][3] = 14.0

In [86]:
import pickle

with open('scores_dict_reduced.pkl', 'rb') as file:
    scores_dict = pickle.load( file)

In [14]:
scores_dict = {}

# Iterate over each point cloud and its corresponding mesh
for (pc, mesh), pc_index in zip(zip(pcds_o3d, meshes), range(len(pcds_o3d))):
    pc_name = stl_file_names[pc_index]  # Adjust the split if needed
    pc_scores = []  # Initialize scores as an empty list

    shape_index = 0
    for shape_path, shape in rep_shapes_dict.items():
        scores = compute_similarity(pc, shape, mesh, int(shape_path))
        # scores = compute_similarity(pc, shape, int(shape_path))  
        
        # Append each score in the rmse list to pc_scores
        pc_scores.extend(scores)  # Assuming scores is a list of rmse values
        shape_index += 1
        # Print results
        print(f"PointCloud {pc_name} is done for shape {shape_index} with the score: {scores}")
    # Store the scores for the current point cloud in the main dictionary
    scores_dict[pc_name] = pc_scores

# [mesh_center_mean, distance_normal, angle_normal,euclidean_norm_std,manhattan_norm_std,euclidean_norm_mean,manhattan_norm_mean, hausdorff, rmse, centre_std, average_density, normal_var_x, normal_var_y, std_fpfh_std, mean_fpfh_std]


NameError: name 'rep_shapes_dict' is not defined

In [ ]:
directory_path = stl_directory
visualize_mesh_with_triangles[''

In [ ]:
mesh = meshes[892]
mesh.remove_unreferenced_vertices()
mesh.remove_duplicated_triangles()
mesh.remove_duplicated_vertices()
mesh.remove_non_manifold_edges()
mesh.compute_vertex_normals()


# Example usage assuming mesh is already defined
vertices = np.asarray(mesh.vertices)  # Assuming mesh.vertices gives the vertices
triangles = np.asarray(mesh.triangles)  # Assuming mesh.triangles gives the triangles
normals = np.asarray(mesh.vertex_normals)  # Assuming this computes or retrieves vertex normals

# Find adjacent normals
adjacent_normals = find_adjacent_normals(vertices, triangles, normals)

# Classify vertex normals
vertex_categories = classify_vertex_normals(normals, adjacent_normals)



# Update graph construction to include edge lengths
G = nx.Graph()
# edge_lengths = {category: 0 for category in categories}  # Initialize sum of edge lengths for each category
total_edge_length = 0  # Total length of all edges

# Initialize dictionaries to track both edge lengths and edge counts
edge_lengths = {category: 0 for category in categories}  # Total length of edges in each category
edge_counts = {category: 0 for category in categories}  # Count of edges in each category

for i, category in enumerate(vertex_categories):
    G.add_node(i, category=category, position=vertices[i])

for triangle in triangles:
    for i in range(3):
        v1, v2 = triangle[i], triangle[(i+1)%3]
        length = calculate_edge_length(vertices[v1], vertices[v2])
        
        # Check if both vertices share the same category for the edge
        if G.nodes[v1]['category'] == G.nodes[v2]['category']:
            category = G.nodes[v1]['category']
            edge_lengths[category] += length
            edge_counts[category] += 1

# Calculate mean edge lengths for each category
mean_edge_lengths = {category: (edge_lengths[category] / edge_counts[category] if edge_counts[category] > 0 else 0) for category in categories}


print("Mean edge length by category:", mean_edge_lengths['concave'], mean_edge_lengths['convex'])


In [ ]:
Mean edge length by category: 1.3061658356233667 2.4289997444434097

1.542693182316484 2.0101829144524146

In [ ]:
2.1876317847300966 3.660512155641339

2.0622376389827246 3.6677787477296686

In [ ]:
017_2020_Teklifid_286.stl: matching file 017_2020_Teklifid_290.stl is beyond top 5, ranked at 18
099_2022_Teklifid_4141.stl: matching file 099_2022_Teklifid_4131.stl is beyond top 5, ranked at 65
036_2020_Teklifid_404.stl: matching file 309_2021_Teklifid_3029.stl is beyond top 5, ranked at 6
997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 40
142_2020_Teklifid_1374.stl: matching file 236_2020_Teklifid_1998.stl is beyond top 5, ranked at 42
228_2020_Teklifid_1967.stl: matching file 228_2020_Teklifid_1968.stl is beyond top 5, ranked at 89
543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 70
543_2021_Teklifid_3308.stl: matching file 543_2021_Teklifid_3307.stl is beyond top 5, ranked at 8
997_2021_Teklifid_3924.stl: matching file 997_2021_Teklifid_3940.stl is beyond top 5, ranked at 18
053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 216
093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 105
098_2022_Teklifid_4135.stl: matching file 098_2022_Teklifid_4134.stl is beyond top 5, ranked at 11
543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 20
142_2020_Teklifid_1837.stl: matching file 142_2020_Teklifid_1393.stl is beyond top 5, ranked at 29
007_2021_Teklifid_2365.stl: matching file 007_2021_Teklifid_2337.stl is beyond top 5, ranked at 15
121_2020_Teklifid_1019.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 16
184_2022_Teklifid_4342.stl: matching file 007_2020_Teklifid_186.stl is beyond top 5, ranked at 11
120_2020_Teklifid_994.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 53
037_2021_Teklifid_2457.stl: matching file 099_2022_Teklifid_4133.stl is beyond top 5, ranked at 61
140_2020_Teklifid_1212.stl: matching file 140_2020_Teklifid_1211.stl is beyond top 5, ranked at 18

In [ ]:
stl_file_names.index('228_2020_Teklifid_1967.stl')

In [ ]:
stl_file_names.index('228_2020_Teklifid_1968.stl')


In [ ]:
stl_file_names.index('099_2022_Teklifid_4131.stl')


In [ ]:
scores_dict = modified_dict

In [88]:
## Normalizing the scores in scores_dict
def normalize_scores(scores_dict):
    # Convert scores to numpy array for normalization
    scores_array = np.array(list(scores_dict.values()))

    # Create a MinMaxScaler
    scaler = MinMaxScaler()

    # Fit and transform the data
    normalized_scores = scaler.fit_transform(scores_array)

    # Convert normalized scores back to the dictionary format
    normalized_scores_dict = {key: list(value) for key, value in zip(scores_dict.keys(), normalized_scores)}

    return normalized_scores_dict

def create_ranking_dict(distance_dict):
    ranking_dict = {}

    for pc_name, distances in distance_dict.items():
        ranking = [name for name, _ in distances]
        ranking_dict[pc_name] = ranking

    return ranking_dict


# Function to compute Manhattan distances
def compute_manhattan_distances(scores_dict):
    distance_dict = {}

    for pc_name, pc_scores in scores_dict.items():
        distances = []
        for other_pc_name, other_pc_scores in scores_dict.items():
            if pc_name != other_pc_name:
                # Calculate Manhattan distance
                distance = cityblock(pc_scores, other_pc_scores)
                distances.append((other_pc_name, distance))
        # Sort based on distance
        distances.sort(key=lambda x: x[1])
        distance_dict[pc_name] = distances

    return distance_dict
    


def compute_euclidean_distances(scores_dict):
    distance_dict = {}

    for pc_name, pc_scores in scores_dict.items():
        distances = []
        for other_pc_name, other_pc_scores in scores_dict.items():
            if pc_name != other_pc_name:
                # Calculate Euclidean distance
                distance = euclidean(pc_scores, other_pc_scores)
                distances.append((other_pc_name, distance))
        # Sort based on distance
        distances.sort(key=lambda x: x[1])
        distance_dict[pc_name] = distances

    return distance_dict


# Your existing code to normalize and rank the scores
normalized_dict = normalize_scores(scores_dict)

# Compute Manhattan distances
# distance_dict = compute_manhattan_distances(scores_dict)
distance_dict = compute_euclidean_distances(normalized_dict)

# Create the ranking dictionary
ranking_dict = create_ranking_dict(distance_dict)

a = ''
# Read the Excel file
curr=os.getcwd()

df = pd.read_excel(curr+'\\labelled_files.xlsx')

# Loop through the dataframe
for index, row in df.iterrows():
    stl_file_name = row['part_id_stl_file']
    top_choice = row['top_choice_stl_file']

    # Check if stl_file_name is in ranking_dict
    if stl_file_name in ranking_dict:
        ranking = ranking_dict[stl_file_name]
        
        # Check if the top choice matches the first rank in ranking_dict
        if ranking[0] == top_choice:
            # print(f"{stl_file_name}: matching file {top_choice} has rank 1")
            pass
        else:
            
            # Find the rank of the top choice in the ranking list
            if top_choice in ranking:
                rank = ranking.index(top_choice) + 1
                if rank <= 5:
                    # print(f"{stl_file_name}: matching file {top_choice} has rank {rank}")
                    pass
                else:
                    print(f"{stl_file_name}: matching file {top_choice} is beyond top 5, ranked at {rank}")
                    pass
            else:
                pass
                # print(f"{stl_file_name}: matching file {top_choice} is not ranked")
    else:
        print(f"{stl_file_name} is not in the ranking dictionary")

#ALIGNMENT AND VISUALIZATION FUNCTIONS 

def quaternion_from_vectors(vec1, vec2):
    """ Create a quaternion representing the rotation from vec1 to vec2 """
    v1_norm = vec1 / np.linalg.norm(vec1)
    v2_norm = vec2 / np.linalg.norm(vec2)
    d = np.dot(v1_norm, v2_norm)

    if np.isclose(d, -1.0):
        # Vectors are nearly opposite
        axis = np.cross(v1_norm, np.array([1, 0, 0]))
        if np.linalg.norm(axis) < 1e-8:  # If parallel to x-axis, use y-axis
            axis = np.cross(v1_norm, np.array([0, 1, 0]))
        axis = axis / np.linalg.norm(axis)
        return o3d.geometry.get_rotation_matrix_from_axis_angle(np.pi * axis)

    s = np.sqrt((1 + d) * 2)
    axis = np.cross(v1_norm, v2_norm)
    q = np.array([s * 0.5, axis[0] / s, axis[1] / s, axis[2] / s])
    return o3d.geometry.get_rotation_matrix_from_quaternion(q)

def align_mesh(mesh, target_axis1=[0, 0, 1], target_axis2=[0, 1, 0], target_axis3=[1, 0, 0]):
    vertices = np.asarray(mesh.vertices)
    centroid = vertices.mean(axis=0)
    centered_vertices = vertices - centroid

    try:
        u, s, vh = np.linalg.svd(centered_vertices, full_matrices=False)
    except np.linalg.LinAlgError:
        print("SVD did not converge, skipping alignment for this mesh.")
        return mesh

    # Align the first principal axis
    principal_axis1 = vh[0, :]
    rotation_matrix1 = quaternion_from_vectors(principal_axis1, target_axis1)
    rotated_vertices = np.dot(centered_vertices, rotation_matrix1.T)

    # Recalculate the principal axes and align the second principal axis
    u, s, vh = np.linalg.svd(rotated_vertices, full_matrices=False)
    principal_axis2 = vh[1, :]
    rotation_matrix2 = quaternion_from_vectors(principal_axis2, target_axis2)
    rotated_vertices = np.dot(rotated_vertices, rotation_matrix2.T)

    # Recalculate the principal axes again and align the third principal axis
    u, s, vh = np.linalg.svd(rotated_vertices, full_matrices=False)
    principal_axis3 = vh[2, :]
    rotation_matrix3 = quaternion_from_vectors(principal_axis3, target_axis3)
    rotated_vertices = np.dot(rotated_vertices, rotation_matrix3.T)

    mesh.vertices = o3d.utility.Vector3dVector(rotated_vertices + centroid)
    # mesh = final_orientation_adjustment(mesh, vertex_index=0, target_direction=[1 ,0, 0])
    mesh = translate_mesh_to_origin(mesh)

 
    return mesh

def visualize_mesh_with_triangles(mesh):
    """Visualizes a mesh with colored triangles and black lines along the edges of the triangles."""

    # mesh = translate_mesh_to_origin(mesh)
    vertex_colors = np.full((len(np.asarray(mesh.vertices)), 3), [0.8, 0.8, 0.8])
    mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)
    lines = [[triangle[0], triangle[1]] for triangle in np.asarray(mesh.triangles)] + \
            [[triangle[1], triangle[2]] for triangle in np.asarray(mesh.triangles)] + \
            [[triangle[2], triangle[0]] for triangle in np.asarray(mesh.triangles)]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(np.asarray(mesh.vertices)),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.paint_uniform_color([0, 0, 0])
    return [mesh, line_set]

def visualize_meshes(filenames, directory_path):
    geometries_to_show = []
    current_x_position = 50
    spacing_factor = 2.0  # Adjust this factor to control spacing between the meshes

    for i, filename in enumerate(filenames):
        full_path = os.path.join(directory_path, filename)
        mesh_open3d = o3d.io.read_triangle_mesh(full_path)
        mesh_open3d = align_mesh(mesh_open3d)
        # Compute the axis-aligned bounding box to get the x dimension
        aabb = mesh_open3d.get_axis_aligned_bounding_box()
        x_dim = aabb.get_extent()[0]
        # For the first mesh (target mesh), we start at (0,0,0) and account for its size in spacing
        if i == 0:
            mesh_open3d.translate(-aabb.get_center())
        else:
            mesh_open3d.translate((current_x_position, 0, 0) - aabb.get_center())
        geometries_to_show.extend(visualize_mesh_with_triangles(mesh_open3d))
        current_x_position += x_dim * spacing_factor  # Update the x position
    
    o3d.visualization.draw_geometries(geometries_to_show)

#ALIGNMENT AND VISUALIZATION FUNCTIONS 

def quaternion_from_vectors(vec1, vec2):
    """ Create a quaternion representing the rotation from vec1 to vec2 """
    v1_norm = vec1 / np.linalg.norm(vec1)
    v2_norm = vec2 / np.linalg.norm(vec2)
    d = np.dot(v1_norm, v2_norm)

    if np.isclose(d, -1.0):
        # Vectors are nearly opposite
        axis = np.cross(v1_norm, np.array([1, 0, 0]))
        if np.linalg.norm(axis) < 1e-8:  # If parallel to x-axis, use y-axis
            axis = np.cross(v1_norm, np.array([0, 1, 0]))
        axis = axis / np.linalg.norm(axis)
        return o3d.geometry.get_rotation_matrix_from_axis_angle(np.pi * axis)

    s = np.sqrt((1 + d) * 2)
    axis = np.cross(v1_norm, v2_norm)
    q = np.array([s * 0.5, axis[0] / s, axis[1] / s, axis[2] / s])
    return o3d.geometry.get_rotation_matrix_from_quaternion(q)

def align_mesh(mesh, target_axis1=[0, 0, 1], target_axis2=[0, 1, 0], target_axis3=[1, 0, 0]):
    vertices = np.asarray(mesh.vertices)
    centroid = vertices.mean(axis=0)
    centered_vertices = vertices - centroid

    try:
        u, s, vh = np.linalg.svd(centered_vertices, full_matrices=False)
    except np.linalg.LinAlgError:
        print("SVD did not converge, skipping alignment for this mesh.")
        return mesh

    # Align the first principal axis
    principal_axis1 = vh[0, :]
    rotation_matrix1 = quaternion_from_vectors(principal_axis1, target_axis1)
    rotated_vertices = np.dot(centered_vertices, rotation_matrix1.T)

    # Recalculate the principal axes and align the second principal axis
    u, s, vh = np.linalg.svd(rotated_vertices, full_matrices=False)
    principal_axis2 = vh[1, :]
    rotation_matrix2 = quaternion_from_vectors(principal_axis2, target_axis2)
    rotated_vertices = np.dot(rotated_vertices, rotation_matrix2.T)

    # Recalculate the principal axes again and align the third principal axis
    u, s, vh = np.linalg.svd(rotated_vertices, full_matrices=False)
    principal_axis3 = vh[2, :]
    rotation_matrix3 = quaternion_from_vectors(principal_axis3, target_axis3)
    rotated_vertices = np.dot(rotated_vertices, rotation_matrix3.T)

    mesh.vertices = o3d.utility.Vector3dVector(rotated_vertices + centroid)
    # mesh = final_orientation_adjustment(mesh, vertex_index=0, target_direction=[1 ,0, 0])
    mesh = translate_mesh_to_origin(mesh)

 
    return mesh

#VISUALIZE BAD EXAMPLES, THE FIRST ONE IS THE TARGET PART, SECOND ONE IS THE ERMETAL TOP CHOICE, AND THE REST ARE ALGORITHM CHOICES
current_dir = os.getcwd()
# loading ermetal stl meshes
# stl_directory = os.path.join(current_dir,'updated_stl')
stl_directory = os.path.join(current_dir,'ermetal')

pcd_directory = os.path.join(current_dir,'pcd_5k')
# Initialize a list to store filenames for visualization along with their top choices
files_to_visualize_with_top_choice = []

# Loop through the dataframe
for index, row in df.iterrows():
    stl_file_name = row['part_id_stl_file']
    top_choice = row['top_choice_stl_file']

    if stl_file_name in ranking_dict:
        ranking = ranking_dict[stl_file_name]

        if ranking[0] != top_choice and top_choice in ranking:
            rank = ranking.index(top_choice) + 1
            if rank > 5:
                # Add the STL file, its top choice, and its top 5 similar parts to the visualization list
                files_to_visualize_with_top_choice.append((stl_file_name, top_choice, ranking[:3]))
    else:
        print(f"{stl_file_name} is not in the ranking dictionary")

# Visualize the collected STL files, their top choices, and their top 5 similar parts
directory_path = stl_directory  # Replace with the actual path to your STL files
for stl_file, top_choice, top_matches in files_to_visualize_with_top_choice:
    print('target file',stl_file,': ',scores_dict[stl_file],"\n")
    print('ermetal top choice', top_choice, ": ", scores_dict[top_choice],'\n')

    # print('target file',stl_file,': ',normalized_dict[stl_file],"\n")
    # print('ermetal top choice', top_choice, ": ", normalized_dict[top_choice],'\n')
    
    for i in range(len(top_matches)):
        print('algorithm top match #',i+1, top_matches[i],": ",scores_dict[top_matches[i]],'\n')
        # print('algorithm top match #',i+1, top_matches[i],": ",normalized_dict[top_matches[i]],'\n')
        
    visualize_filenames = [stl_file, top_choice] + top_matches
    visualize_meshes(visualize_filenames, directory_path)


# 099_2022_Teklifid_4141.stl: matching file 099_2022_Teklifid_4131.stl is beyond top 5, ranked at 72
# 228_2020_Teklifid_1921.stl: matching file 228_2020_Teklifid_1922.stl is beyond top 5, ranked at 6
# 036_2020_Teklifid_404.stl: matching file 309_2021_Teklifid_3029.stl is beyond top 5, ranked at 9
# 997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 18
# 997_2021_Teklifid_3882.stl: matching file 997_2021_Teklifid_3915.stl is beyond top 5, ranked at 6
# 142_2020_Teklifid_1374.stl: matching file 236_2020_Teklifid_1998.stl is beyond top 5, ranked at 83
# 228_2020_Teklifid_1967.stl: matching file 228_2020_Teklifid_1968.stl is beyond top 5, ranked at 71
# 003_2021_Teklifid_2318.stl: matching file 003_2021_Teklifid_2319.stl is beyond top 5, ranked at 14
# 228_2020_Teklifid_1921.stl: matching file 228_2020_Teklifid_1922.stl is beyond top 5, ranked at 6
# 543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 46
# 997_2021_Teklifid_3924.stl: matching file 997_2021_Teklifid_3940.stl is beyond top 5, ranked at 15
# 053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 179
# 093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 114
# 098_2022_Teklifid_4135.stl: matching file 098_2022_Teklifid_4134.stl is beyond top 5, ranked at 17
# 543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 16
# 142_2020_Teklifid_1837.stl: matching file 142_2020_Teklifid_1393.stl is beyond top 5, ranked at 30
# 997_2021_Teklifid_3940.stl: matching file 997_2021_Teklifid_3932.stl is beyond top 5, ranked at 6
# 121_2020_Teklifid_1019.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 16
# 120_2020_Teklifid_994.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 49
# 037_2021_Teklifid_2457.stl: matching file 099_2022_Teklifid_4133.stl is beyond top 5, ranked at 48
# 140_2020_Teklifid_1212.stl: matching file 140_2020_Teklifid_1211.stl is beyond top 5, ranked at 16


# 017_2020_Teklifid_286.stl: matching file 017_2020_Teklifid_290.stl is beyond top 5, ranked at 23
# 099_2022_Teklifid_4141.stl: matching file 099_2022_Teklifid_4131.stl is beyond top 5, ranked at 51
# 036_2020_Teklifid_404.stl: matching file 309_2021_Teklifid_3029.stl is beyond top 5, ranked at 7
# 997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 18
# 142_2020_Teklifid_1374.stl: matching file 236_2020_Teklifid_1998.stl is beyond top 5, ranked at 69
# 228_2020_Teklifid_1967.stl: matching file 228_2020_Teklifid_1968.stl is beyond top 5, ranked at 61
# 543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 43
# 997_2021_Teklifid_3924.stl: matching file 997_2021_Teklifid_3940.stl is beyond top 5, ranked at 14
# 053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 170
# 093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 106
# 098_2022_Teklifid_4135.stl: matching file 098_2022_Teklifid_4134.stl is beyond top 5, ranked at 12
# 543_2021_Teklifid_3251.stl: matching file 008_2021_Teklifid_2332.stl is beyond top 5, ranked at 22
# 142_2020_Teklifid_1837.stl: matching file 142_2020_Teklifid_1393.stl is beyond top 5, ranked at 26
# 007_2021_Teklifid_2365.stl: matching file 007_2021_Teklifid_2337.stl is beyond top 5, ranked at 20
# 997_2021_Teklifid_3940.stl: matching file 997_2021_Teklifid_3932.stl is beyond top 5, ranked at 6
# 121_2020_Teklifid_1019.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 11
# 120_2020_Teklifid_994.stl: matching file 120_2020_Teklifid_989.stl is beyond top 5, ranked at 45
# 037_2021_Teklifid_2457.stl: matching file 099_2022_Teklifid_4133.stl is beyond top 5, ranked at 49
# 140_2020_Teklifid_1212.stl: matching file 140_2020_Teklifid_1211.stl is beyond top 5, ranked at 16

# 099_2022_Teklifid_4141.stl: matching file 099_2022_Teklifid_4131.stl is beyond top 5, ranked at 8
# 228_2020_Teklifid_1967.stl: matching file 228_2020_Teklifid_1968.stl is beyond top 5, ranked at 10
# 543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 8
# 053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 22
# 093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 14
# 037_2021_Teklifid_2457.stl: matching file 099_2022_Teklifid_4133.stl is beyond top 5, ranked at 12
# 140_2020_Teklifid_1212.stl: matching file 140_2020_Teklifid_1211.stl is beyond top 5, ranked at 6

017_2020_Teklifid_286.stl: matching file 017_2020_Teklifid_290.stl is beyond top 5, ranked at 20
063_2020_Teklifid_732.stl: matching file 063_2020_Teklifid_733.stl is beyond top 5, ranked at 8
121_2020_Teklifid_1034.stl: matching file 121_2020_Teklifid_1035.stl is beyond top 5, ranked at 17
036_2020_Teklifid_404.stl: matching file 309_2021_Teklifid_3029.stl is beyond top 5, ranked at 9
997_2021_Teklifid_3933.stl: matching file 997_2021_Teklifid_3938.stl is beyond top 5, ranked at 33
029_2020_Teklifid_380.stl: matching file 029_2020_Teklifid_381.stl is beyond top 5, ranked at 7
543_2021_Teklifid_3322.stl: matching file 543_2021_Teklifid_3323.stl is beyond top 5, ranked at 278
997_2021_Teklifid_3924.stl: matching file 997_2021_Teklifid_3940.stl is beyond top 5, ranked at 610
053_2022_Teklifid_3982.stl: matching file 053_2022_Teklifid_3983.stl is beyond top 5, ranked at 13
093_2020_Teklifid_804.stl: matching file 184_2022_Teklifid_4375.stl is beyond top 5, ranked at 64
004_2020_Teklifid_3

KeyboardInterrupt: 